# Interview processing and insertion into test database

In [1]:
import psycopg2
import getpass
import os
import docx2json
import json
import copy
import nltk.tokenize
import nltk.corpus
import nltk.stem

## Setting up database connection

In [34]:
help(nltk.tokenize)

Help on package nltk.tokenize in nltk:

NAME
    nltk.tokenize - NLTK Tokenizer Package

DESCRIPTION
    Tokenizers divide strings into lists of substrings.  For example,
    tokenizers can be used to find the words and punctuation in a string:
    
        >>> from nltk.tokenize import word_tokenize
        >>> s = '''Good muffins cost $3.88\nin New York.  Please buy me
        ... two of them.\n\nThanks.'''
        >>> word_tokenize(s)
        ['Good', 'muffins', 'cost', '$', '3.88', 'in', 'New', 'York', '.',
        'Please', 'buy', 'me', 'two', 'of', 'them', '.', 'Thanks', '.']
    
    This particular tokenizer requires the Punkt sentence tokenization
    models to be installed. NLTK also provides a simpler,
    regular-expression based tokenizer, which splits text on whitespace
    and punctuation:
    
        >>> from nltk.tokenize import wordpunct_tokenize
        >>> wordpunct_tokenize(s)
        ['Good', 'muffins', 'cost', '$', '3', '.', '88', 'in', 'New', 'York', '.',
     

In [2]:
conn = psycopg2.connect("host=%s port=%s dbname=%s user=%s password=%s" 
                        % ((str(input("Host: "))).strip(), (str(input("Port: "))).strip(), (str(input("DatabaseName: "))).strip(), 
                           (str(input("User: "))).strip(), getpass.getpass("Password: ")))
cur = conn.cursor()
cur.close()

Host:  localhost
Port:  5432
DatabaseName:  icgeo
User:  andre
Password:  ···········


## Processing interviews with simple pattern recognition / standardizing

### Listing input files

In [3]:
# Get all input files and sort by athlete id
input_files = sorted(os.listdir('./source-files/GRD Pronto/Entrevistas Completas'), key=lambda id: int(id[:2]))
input_files_ids = [int(f[:2]) for f in input_files]
input_files = [os.getcwd() + '/source-files/GRD Pronto/Entrevistas Completas/' + file for file in input_files]
print("Interviews (number of documents = %d):" % len(input_files), input_files_ids, *input_files, sep='\n')

Interviews (number of documents = 18):
[19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 41, 42]
/home/andre/Documents/ic-geo/refactored/source-files/GRD Pronto/Entrevistas Completas/19 - Camila Ferezin do Amarante ( Ginastica ritmica ).docx
/home/andre/Documents/ic-geo/refactored/source-files/GRD Pronto/Entrevistas Completas/20 - Natália Scherer Eidt (Ginastica Ritmica) .docx
/home/andre/Documents/ic-geo/refactored/source-files/GRD Pronto/Entrevistas Completas/21 - Flavia Cristina de Faria (Ginastica Ritmica).docx
/home/andre/Documents/ic-geo/refactored/source-files/GRD Pronto/Entrevistas Completas/22 - Alessandra Ferezin Guidugli (Ginástica Rítmica).docx
/home/andre/Documents/ic-geo/refactored/source-files/GRD Pronto/Entrevistas Completas/23 - Thalita Nakadomari (Ginastica  Ritmica).docx
/home/andre/Documents/ic-geo/refactored/source-files/GRD Pronto/Entrevistas Completas/24 -  Dayane Camilo.docx
/home/andre/Documents/ic-geo/refactored/source-files/GRD Pronto/Entrevist

### Converting input files to JSON with docx2json

In [4]:
# Convert all documents into a JSON (dict) list
json_arr = [json.loads(docx2json.convert(f)) for f in input_files]

### Simple pattern recognition

In [5]:
# Separating pattern indexes
# I may need to make those recognitions more complex, depending on new patterns and possible match in existing ones
json_patterns = {}
json_patterns['bold-nonbold'] = [idx for idx, f in enumerate(json_arr) if f['text'][0] == 'Início da transcrição']
json_patterns['athlete-name'] = [idx for idx, f in enumerate(json_arr) if 'Atleta: ' in f['text'][0]]
json_patterns['one-one-abbr'] = [idx for idx, f in enumerate(json_arr) if 'USP – ' in f['text'][1] or 'USP – ' in f['text'][2]]
json_patterns['black-colored'] = [idx for idx, f in enumerate(json_arr) if (' nascida em ' in f['text'][0] or ' nascido em ' in f['text'][0])]
json_patterns['all-nonbold-names'] = [idx for idx, f in enumerate(json_arr) if 'Transcrição' in f['text'][0]]
json_patterns

{'bold-nonbold': [0, 2, 4, 5, 6, 8, 10, 11, 12, 13, 14, 15, 16],
 'athlete-name': [1, 3],
 'one-one-abbr': [7],
 'black-colored': [9],
 'all-nonbold-names': [17]}

### Processing each pattern

In [6]:
# Processing 'bold-nonbold' pattern files
# Removing 'Início da transcrição' from 'bold-nonbold' pattern files
for idx in json_patterns['bold-nonbold']:
    if json_arr[idx]['text'][0] == 'Início da transcrição':
        del json_arr[idx]['text'][0]
    if json_arr[idx]['bold'][0] == 'Início da transcrição':
        del json_arr[idx]['bold'][0]
    if 'Início da transcrição ' in json_arr[idx]['bold'][0]:
        json_arr[idx]['bold'][0] = json_arr[idx]['bold'][0].replace('Início da transcrição ', '')
#   Removing trailing spaces
    for idx2, s in enumerate(json_arr[idx]['text']):
        json_arr[idx]['text'][idx2] = s.strip(' ')
    for idx2, s in enumerate(json_arr[idx]['bold']):
        json_arr[idx]['bold'][idx2] = s.strip(' ')
    for idx2, s in enumerate(json_arr[idx]['nonbold']):
        json_arr[idx]['nonbold'][idx2] = s.strip(' ')

In [7]:
# Processing 'athlete-name' pattern files
# Removing names from paragraph authors and trailing spaces, along other redundant information
questionset = set(['Kátia', 'Entrevistador']) # to determine if the paragraph is 'bold' or 'nonbold'
for idx in json_patterns['athlete-name']:
    if 'Atleta: ' in json_arr[idx]['text'][0]:
        del json_arr[idx]['text'][0:2]
        json_arr[idx]['nonbold'] = []
        for idx2, s in enumerate(json_arr[idx]['text']):
            json_arr[idx]['text'][idx2] = s.strip(' ')
            splitpar = s.split(': ', 1)
            if (len(splitpar)==2):
                json_arr[idx]['text'][idx2] = splitpar[1]
                if (splitpar[0] in questionset):
                    json_arr[idx]['bold'].append(splitpar[1])
                else:
                    json_arr[idx]['nonbold'].append(splitpar[1])
            else:
                json_arr[idx]['nonbold'].append(splitpar[0])
#   Removing trailing spaces
    for idx2, s in enumerate(json_arr[idx]['text']):
        json_arr[idx]['text'][idx2] = s.strip(' ')
    for idx2, s in enumerate(json_arr[idx]['bold']):
        json_arr[idx]['bold'][idx2] = s.strip(' ')
    for idx2, s in enumerate(json_arr[idx]['nonbold']):
        json_arr[idx]['nonbold'][idx2] = s.strip(' ')

In [8]:
# Processing 'one-one-abbr' pattern files
# Removing names from paragraph authors and trailing spaces, along other redundant information
questionset = set(['USP'])
for idx in json_patterns['one-one-abbr']:
    if ('USP – ' in json_arr[idx]['text'][1] or 'USP – ' in json_arr[idx]['text'][2]):
        del json_arr[idx]['text'][0]
        json_arr[idx]['nonbold'] = []
        for idx2, s in enumerate(json_arr[idx]['text']):
            json_arr[idx]['text'][idx2] = s.strip(' ')
            splitpar = s.split(' – ', 1)
            if (len(splitpar)==2):
                json_arr[idx]['text'][idx2] = splitpar[1]
                if (splitpar[0] in questionset):
                    json_arr[idx]['bold'].append(splitpar[1])
                else:
                    json_arr[idx]['nonbold'].append(splitpar[1])
            else:
                json_arr[idx]['nonbold'].append(splitpar[0])
#   Removing trailing spaces
    for idx2, s in enumerate(json_arr[idx]['text']):
        json_arr[idx]['text'][idx2] = s.strip(' ')
    for idx2, s in enumerate(json_arr[idx]['bold']):
        json_arr[idx]['bold'][idx2] = s.strip(' ')
    for idx2, s in enumerate(json_arr[idx]['nonbold']):
        json_arr[idx]['nonbold'][idx2] = s.strip(' ')

In [9]:
# Processing 'black-colored' pattern files
check = False
for idx in json_patterns['black-colored']:
    json_arr[idx]['nonbold'] = []
    json_arr[idx]['bold'] = []
# Removing unwanted information
    for idx2, s in enumerate(json_arr[idx]['text']):
        if (not check):
            check = 'Entrevista realizada em ' in s
        else:
            del json_arr[idx]['text'][0:idx2]
            break
# Separating questions and answers
    for idx2, s in enumerate(json_arr[idx]['text']):
        if (idx2 % 2 == 0):
            json_arr[idx]['bold'].append(s)
        else:
            json_arr[idx]['nonbold'].append(s)
# Removing trailing spaces
    for idx2, s in enumerate(json_arr[idx]['text']):
        json_arr[idx]['text'][idx2] = s.strip(' ')
    for idx2, s in enumerate(json_arr[idx]['bold']):
        json_arr[idx]['bold'][idx2] = s.strip(' ')
    for idx2, s in enumerate(json_arr[idx]['nonbold']):
        json_arr[idx]['nonbold'][idx2] = s.strip(' ')

In [10]:
# Processing 'all-nonbold-names' pattern files

# Processing 'athlete-name' pattern files
# Removing names from paragraph authors and trailing spaces, along other redundant information
questionset = set(['Kátia', 'Entrevistador', 'ENTREVISTADORA']) # to determine if the paragraph is 'bold' or 'nonbold'
for idx in json_patterns['all-nonbold-names']:
    if 'Transcrição ' in json_arr[idx]['text'][0]:
        del json_arr[idx]['text'][0]
        json_arr[idx]['nonbold'] = []
        for idx2, s in enumerate(json_arr[idx]['text']):
            json_arr[idx]['text'][idx2] = s.strip(' ')
            splitpar = s.split(': ', 1)
            if (len(splitpar)==2):
                json_arr[idx]['text'][idx2] = splitpar[1]
                if (splitpar[0] in questionset):
                    json_arr[idx]['bold'].append(splitpar[1])
                else:
                    json_arr[idx]['nonbold'].append(splitpar[1])
            else:
                json_arr[idx]['nonbold'].append(splitpar[0])
#   Removing trailing spaces
    for idx2, s in enumerate(json_arr[idx]['text']):
        json_arr[idx]['text'][idx2] = s.strip(' ')
    for idx2, s in enumerate(json_arr[idx]['bold']):
        json_arr[idx]['bold'][idx2] = s.strip(' ')
    for idx2, s in enumerate(json_arr[idx]['nonbold']):
        json_arr[idx]['nonbold'][idx2] = s.strip(' ')

## Generating META Information

In [35]:
# Duplicate, remove punctation and split into single words for each interview
# Create set of tokens for each interview, removing stopwords in the process
interviews_split = []
tokens = []
for idx, x in enumerate(json_arr):
    interviews_split.append({
        'text': nltk.tokenize.word_tokenize('\n'.join(x['text']).lower(), language='portuguese'),
        'q': nltk.tokenize.word_tokenize('\n'.join(x['bold']).lower(), language='portuguese'),
        'a': nltk.tokenize.word_tokenize('\n'.join(x['nonbold']).lower(), language='portuguese')
    })
    tokens.append({
        'text': set(interviews_split[idx]['text']) - set(nltk.corpus.stopwords.words('portuguese')),
        'q': set(interviews_split[idx]['q']) - set(nltk.corpus.stopwords.words('portuguese')),
        'a': set(interviews_split[idx]['a']) - set(nltk.corpus.stopwords.words('portuguese'))
    })

In [39]:
# Generating bag-of-words (stemmed or not) for insertion at the database
stemmer = nltk.stem.snowball.SnowballStemmer('portuguese')
# stemmer = nltk.stem.rslp.RSLPStemmer()
metas = []
for idx, x in enumerate(interviews_split):
    meta = {'text': {'bag': {}, 'bag_stemmed': {}}, 
                'q': {'bag': {}, 'bag_stemmed': {}}, 
                'a': {'bag': {}, 'bag_stemmed': {}}}
    for token in tokens[idx]['text']:
        meta['text']['bag'][token] = x['text'].count(token)
        meta['text']['bag_stemmed'][stemmer.stem(token)] = meta['text']['bag_stemmed'].get(stemmer.stem(token), 0) + meta['text']['bag'][token]
    for token in tokens[idx]['q']:
        meta['q']['bag'][token] = x['q'].count(token)
        meta['q']['bag_stemmed'][stemmer.stem(token)] = meta['q']['bag_stemmed'].get(stemmer.stem(token), 0) + meta['q']['bag'][token]
    for token in tokens[idx]['a']:
        meta['a']['bag'][token] = x['a'].count(token)
        meta['a']['bag_stemmed'][stemmer.stem(token)] = meta['a']['bag_stemmed'].get(stemmer.stem(token), 0) + meta['a']['bag'][token]
    metas.append(meta)

## Insertion of data into the database

In [27]:
cur = conn.cursor()
for idx, data in enumerate(json_arr):
    cur.execute("""INSERT INTO interviews (id, texto, perguntas, respostas) VALUES (%(id)s, %(texto)s, %(perguntas)s, %(respostas)s) ON CONFLICT DO NOTHING;""",
               {'id': input_files_ids[idx], 
                'texto': '\n'.join(data['text']), 
                'perguntas': data['bold'],
                'respostas': data['nonbold']
               })
conn.commit()
cur.close()

## Setting up database connection for performance testing

## Testing full-text search tools and comparing META information

In [31]:
cur = conn.cursor()
cur.execute("""SELECT ID, tsvector_to_array(TO_TSVECTOR('portuguese', texto)) FROM interviews ORDER BY ID ASC;""")
dbout = cur.fetchall()
cur.close()

In [40]:
# Differences between my extraction method and the one done by the DBMS
for idx, i in enumerate(dbout):
    print("ID: ", i[0])
    print("Lista A (in postgres, not in metas):")
    for j in i[1]:
        if j not in list(metas[idx]['text']['bag_stemmed'].keys()):
            print(j)
    print("Lista B (in metas, not in postgres):")
    for j in list(metas[idx]['text']['bag_stemmed'].keys()):
        if j not in i[1]:
            print(j)
    print("")

ID:  19
Lista A (in postgres, not in metas):
06
1
2004
60
68
infant
vic
Lista B (in metas, not in postgres):
:
...
1,68
``
06:24
)
%
,
é.
”
1,60
.
“
91.
2004.
(
vice-
?

ID:  20
Lista A (in postgres, not in metas):
2005
74
74m
american
anti
pré
seguranc
Lista B (in metas, not in postgres):
...
–
2005.
‘
segur
15.
anti-
,
é.
”
.
“
1,74
2003.
’
1,74m
?
17.

ID:  21
Lista A (in postgres, not in metas):
08
14
2.000
41
44
american
inseguranc
lo
pan
vizinhanc
Lista B (in metas, not in postgres):
:
...
82.
)
vizinh
41:44
,
!
14:08
é.
”
.
“
2.000.
97.
(
?
2002.
insegur

ID:  22
Lista A (in postgres, not in metas):
-14
-5
-60
50
lest
lideranc
milk
oest
shak
Lista B (in metas, not in postgres):
:
...
10:30
‘
%
,
50-60
13-14
”
.
alucinando.
“
4-5
lider
2004.
’
?

ID:  23
Lista A (in postgres, not in metas):
..
16
comilanc
esperanc
execuçã
Lista B (in metas, not in postgres):
:
...
no..
e´muit
15,16,17,18
execu
comil
não..
,
!
é.
.
me..
e.
faziam..
2003.
?
ai..
7.

ID:  24
Lista A (in postgres, no